Here we split the data into training, cross-validation, and testing datasets

In [4]:

import numpy as np
import random
import pandas as pd

import re

from sklearn.utils import shuffle

import blurbs
from blurbs import BlurbFeatures
import imp
import feature_generation
from feature_generation import CrossTermComputer, ZipcodeEncoder

imp.reload(feature_generation)
imp.reload(blurbs)


<module 'blurbs' from '/home/bryce/Projects/Data_Science/Apt_Prices/blurbs.py'>

In [2]:
DIR = '/home/bryce/Projects/Data_Science/Apt_Prices/csvs/'

address_file =  DIR + 'pd_address_info.csv' 
apt_file = DIR + 'pd_apt_info.csv'

addr_df = pd.read_csv(address_file, sep=';')
apt_df = pd.read_csv(apt_file, sep=';')

In [16]:
def _extract_lat_and_lng(latlng):
    latlng_list = re.sub(r'[\(\)]', '', latlng).split(',')
    [lat, lng] = map(lambda s : float(s.strip()), latlng_list)
    return (lat, lng)

addrs_and_latlng = addr_df[['address', 'latlng']]
addrs_and_latlng['lat'] = addrs_and_latlng.latlng.map(lambda s : _extract_lat_and_lng(s)[0])
addrs_and_latlng['lng'] = addrs_and_latlng.latlng.map(lambda s : _extract_lat_and_lng(s)[1])
addrs_and_latlng['lat^2'] = addrs_and_latlng.lat ** 2
addrs_and_latlng['lng^2'] = addrs_and_latlng.lng ** 2
addrs_and_latlng['lat * lng'] = addrs_and_latlng.lat * addrs_and_latlng.lng
addrs_and_latlng.drop(columns=['latlng'], inplace=True)
apt_df.sort_values(by='address', inplace=True)
apt_df.reset_index(drop=True, inplace=True)
X = apt_df.merge(addrs_and_latlng, on='address')

/tmp/ipykernel_15340/29860126.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  addrs_and_latlng['lat'] = addrs_and_latlng.latlng.map(lambda s : _extract_lat_and_lng(s)[0])


In [17]:
print (apt_df.tail())
print (X.tail())

                                   address  price  beds  baths   sq_ft  \
1882  99 Westedge St, Charleston, SC 29403   3585     2    2.0  1399.0   
1883  99 Westedge St, Charleston, SC 29403   2985     2    2.0  1053.0   
1884  99 Westedge St, Charleston, SC 29403   1935     1    1.0   648.0   
1885  99 Westedge St, Charleston, SC 29403   4485     2    2.5  1434.0   
1886  99 Westedge St, Charleston, SC 29403   3010     2    2.0   995.0   

                 location    zip  units_in_building  
1882  Westside Charleston  29403                  8  
1883  Westside Charleston  29403                  8  
1884  Westside Charleston  29403                  8  
1885  Westside Charleston  29403                  8  
1886  Westside Charleston  29403                  8  
                                   address  price  beds  baths   sq_ft  \
2039  99 Westedge St, Charleston, SC 29403   1935     1    1.0   648.0   
2040  99 Westedge St, Charleston, SC 29403   4485     2    2.5  1434.0   
2041  99 

In [3]:
# Remove any rows with missing data, and remove data duplicates (e.g. properties listed twice under different management companies)
# Note that since we don't have a reliable way right now of distinguishing between different units from the same property, and the same unit
# listed twice by different property managers, for instance, we might have some duplicate data in apt_df.
# addr_df, on the other hand, is only supposed to contain info about the property. The property doesn't change if someone else lists it, so it should definitely not have duplicates.

print("apt_df shape before: ", apt_df.shape)
print('addr_df shape before: ', addr_df.shape)

# Drop rows with missing data in apt_df
drop_apt_row = apt_df.isna().any(axis=1)
apt_df = apt_df.loc[~drop_apt_row]
apt_df.reset_index(inplace=True, drop=True)

# addr_df shouldn't have missing data
addr_rows_missing_data = addr_df.isna().any(axis=1)
assert(addr_rows_missing_data.sum() == 0)

# But it might have addresses listed twice
addr_df.drop_duplicates(subset=['address'], inplace=True)
addr_df.reset_index(inplace=True, drop=True)


print("apt_df shape after: ", apt_df.shape)
print('addr_df shape after: ', addr_df.shape)

apt_df shape before:  (1887, 8)
addr_df shape before:  (305, 4)
apt_df shape after:  (1873, 8)
addr_df shape after:  (293, 4)


There are two ways we could split the data:

1) Split by units. So many buildings with multiple units will have some units in the training data and some in the CV / testing data
2) Split by buildings. This seems more principled. So we'll start here.

In [49]:
addrs_and_unit = apt_df[['address', 'units_in_building']].drop_duplicates()
addr_with_unit_count = addr_df.merge(addrs_and_unit, on='address').sort_values(by=['units_in_building'])
print(addr_with_unit_count.tail())
num_buildings = addr_with_unit_count.shape[0]

indices = list(range(num_buildings))
random.shuffle(indices)

num_units = apt_df.shape[0]
print(num_units)

                                     address  ... units_in_building
3       211 Satori Way, Charleston, SC 29455  ...                35
11  35 Folly Road Blvd, Charleston, SC 29407  ...                37
65  9345 Blue House Rd, Charleston, SC 29406  ...                37
67     7927 St Ives Rd, Charleston, SC 29406  ...                40
13     838 Morrison Dr, Charleston, SC 29403  ...               279

[5 rows x 5 columns]
1873


In [52]:
# Here we'll split the data into training, validation, and testing sets, using a 70/20/10 split

min_training_units = int(0.7 * num_units)
min_validation_units = int(0.2 * num_units)

addr_with_unit_count = shuffle(addr_with_unit_count)

addr_with_unit_count.reset_index(inplace=True, drop = True)
#print(addr_with_unit_count)

addr_with_unit_count['unit_sum'] = addr_with_unit_count.units_in_building.cumsum()


training_addrs = addr_with_unit_count.loc[addr_with_unit_count.unit_sum <= min_training_units]
validation_addrs = addr_with_unit_count.loc[(addr_with_unit_count.unit_sum > min_training_units) & (addr_with_unit_count.unit_sum <= min_training_units + min_validation_units)].reset_index(drop=True)
testing_addrs = addr_with_unit_count.loc[addr_with_unit_count.unit_sum > (min_training_units + min_validation_units)].reset_index(drop=True)

apt_df_train = pd.merge(apt_df, training_addrs[['address']], on='address')
apt_df_validation = pd.merge(apt_df, validation_addrs[['address']], on='address')
apt_df_test = pd.merge(apt_df, testing_addrs[['address']], on='address')

apt_df_train.to_csv('/tmp/apt_df_train.csv', sep=';', index=False)


In [96]:
# I think this is working now

class FeatureGenerator:
    def __init__(self):
        self.cross_term_computer = None
        self.blurb_features = BlurbFeatures()
        self.zip_encoder = ZipcodeEncoder()
    
    def _merge_svd_feats_and_apt_df(self, svd_df, apt_df, addr_df):
        addrs_and_blurb_svd_feats = pd.concat((addr_df, svd_df), axis = 1)
        addrs_and_blurb_svd_feats.drop(columns=['blurb', 'url', 'latlng', 'units_in_building'], inplace=True)
        addrs_and_blurb_svd_feats.drop_duplicates(subset=['address'], inplace=True)
        addresses_from_apt_df = apt_df[['address']]
        X_blurbs = pd.merge(addrs_and_blurb_svd_feats, addresses_from_apt_df, on='address').sort_values(by=['address']).reset_index(drop=True)
        X_blurbs.drop(columns=['address'], inplace=True)
        return X_blurbs

    def get_training_features(self, apt_df_train, addr_df_train):
        apt_df_train = apt_df_train.sort_values(by=['address']).reset_index(drop=True)
        y = apt_df_train[['price']]
        X = apt_df_train.drop(columns=['price'])

        # 1) generate the zip code dummy variables
        X_zips = self.zip_encoder.generate_training_zipcode_dummy_vars(apt_df_train)

        # 2) Generate the features from the description blurbs
        # fit and generate the SVD features from the blurbs
        self.blurb_features.compute_training_tfidf_matrix(addr_df_train.blurb)
        tfidf_mat = self.blurb_features.get_training_tfidf_matrix()
        print(tfidf_mat.shape)
        svd_df = self.blurb_features.compute_training_svd_df(training_tfidf_matrix=tfidf_mat)

        # these features were generated per-address. now join them with each apartment unit
        X_blurbs = self._merge_svd_feats_and_apt_df(svd_df, apt_df_train, addr_df_train)
        print(X_blurbs.columns)
        X_blurbs.to_csv('/tmp/X_blurbs.csv', sep=';', index=False)
        X = pd.concat((X, X_zips, X_blurbs), axis=1)
        X.to_csv('/tmp/X.csv', sep=';', index=False)

        # 3) Generate the cross-terms
        first_columns = list(X_zips.columns)
        second_columns = ['beds', 'baths', 'sq_ft', 'units_in_building']
        column_pairs = [('sq_ft', 'sq_ft'), ('beds', 'beds'), ('baths', 'baths')]
        self.cross_term_computer = CrossTermComputer(first_columns=first_columns, second_columns=second_columns, column_pairs=column_pairs)
        X_cross_terms = self.cross_term_computer.compute_cross_terms(X)

        X = pd.concat((X, X_cross_terms), axis=1)
        return (X, y)
    
    def get_testing_features(self, apt_df_test, addr_df_test):
        apt_df_test = apt_df_test.sort_values(by=['address']).reset_index(drop=True)
        y = apt_df_test[['price']]
        X0 = apt_df_test.drop(columns=['price'])

        X_zips = self.zip_encoder.generate_testing_zipcode_dummy_vars(apt_df_test)

        svd_df = self.blurb_features.compute_testing_svd_df_from_blurbs(addr_df_test.blurb)
        X_blurbs = self._merge_svd_feats_and_apt_df(svd_df, apt_df_test, addr_df_test)
        X1 = pd.concat((X0, X_zips, X_blurbs), axis=1)

        X_cross_terms = self.cross_term_computer.compute_cross_terms(X1)
        X = pd.concat((X1, X_cross_terms), axis=1)
        return (X, y)



In [88]:
apt_df_train['beds'].dtype

dtype('int64')

In [98]:
print(training_addrs.columns)
feature_generator = FeatureGenerator()
X_train, y_train = feature_generator.get_training_features(apt_df_train=apt_df_train, addr_df_train=training_addrs)
X_test, y_test = feature_generator.get_testing_features(apt_df_test=apt_df_train, addr_df_test=training_addrs)



print("X train, y train shapes", X_train.shape, y_train.shape)
print("X test, y test shapes", X_test.shape, y_test.shape)
print("X_train.head ", X_train.head())
print("X_test.head() ", X_test.head())
equal = X_train.equals(X_test)
print("equal ", equal)


Index(['address', 'latlng', 'url', 'blurb', 'units_in_building', 'unit_sum'], dtype='object')
dim training blurbs, before cleaning (172,)
dim cleaned_blurbs  (172,)
(172, 974)
training svd matrix shape  (172, 30)
about to return the training_svd_df, with shape (172, 30)
Index(['unit_sum', 'svd_1', 'svd_2', 'svd_3', 'svd_4', 'svd_5', 'svd_6',
       'svd_7', 'svd_8', 'svd_9', 'svd_10', 'svd_11', 'svd_12', 'svd_13',
       'svd_14', 'svd_15', 'svd_16', 'svd_17', 'svd_18', 'svd_19', 'svd_20',
       'svd_21', 'svd_22', 'svd_23', 'svd_24', 'svd_25', 'svd_26', 'svd_27',
       'svd_28', 'svd_29', 'svd_30'],
      dtype='object')
columns in dataframe:  Index(['address', 'beds', 'baths', 'sq_ft', 'location', 'zip',
       'units_in_building', 'zip_29401', 'zip_29403', 'zip_29406', 'zip_29407',
       'zip_29412', 'zip_29414', 'zip_29455', 'zip_29464', 'zip_29492',
       'unit_sum', 'svd_1', 'svd_2', 'svd_3', 'svd_4', 'svd_5', 'svd_6',
       'svd_7', 'svd_8', 'svd_9', 'svd_10', 'svd_11', 'sv